#### Import Libraries

In [9]:
#*******________________________Code for Training DSD_Net______________________________*******

import torch
import torch.nn as nn
import os, glob
import random, csv
from torch.autograd import Variable

#import visdom
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from PIL import Image
from tqdm.auto import tqdm
import numpy as np


#### Custom Dataloader

In [10]:
import random
import numpy as np
import torch.utils.data as data
import utils.utils_image as util



class DatasetSR(data.Dataset):
    

    def __init__(self):
        super(DatasetSR, self).__init__()
        self.n_channels = 1
        self.sf = 1
        self.patch_size = 126
        self.L_size = self.patch_size // self.sf

        # ------------------------------------
        # get paths of L/H
        # ------------------------------------
        self.paths_H = util.get_image_paths('/notebooks/JPEG_HR_Gray/DIV2K_train_HR_Grayscale')          #The Path of Ground Truth Images
        self.paths_L = util.get_image_paths('/notebooks/JPEG_LR_Gray/JPEG_Grayscale/Anchor/JPEG_10')     #The path of LR Images 
        '''
        assert self.paths_H, 'Error: H path is empty.'
        if self.paths_L and self.paths_H:
            assert len(self.paths_L) == len(self.paths_H), 'L/H mismatch - {}, {}.'.format(len(self.paths_L), len(self.paths_H))
        '''
    def __getitem__(self, index):

        H_path = None
        # ------------------------------------
        # get H image
        # ------------------------------------
        L_path = self.paths_L[index]
        img_L = util.imread_uint(L_path, self.n_channels)
        img_L = util.uint2single(img_L)

        # ------------------------------------
        # modcrop
        # ------------------------------------
        

        # ------------------------------------
        # get L image
        # ------------------------------------
        if self.paths_H:
            # --------------------------------
            # directly load L image
            # --------------------------------
            H_path = self.paths_H[index % 800]
            img_H = util.imread_uint(H_path, self.n_channels)
            img_H = util.uint2single(img_H)
            img_H = util.modcrop(img_H, self.sf)
        else:
            # --------------------------------
            # sythesize L image via matlab's bicubic
            # --------------------------------
            H, W = img_H.shape[:2]
            img_L = util.imresize_np(img_H, 1 / self.sf, True)

        # ------------------------------------
        # if train, get L/H patch pair
        # ------------------------------------

        H, W, C = img_L.shape

        # --------------------------------
        # randomly crop the L patch
        # --------------------------------
        rnd_h = random.randint(0, max(0, H - self.L_size))
        rnd_w = random.randint(0, max(0, W - self.L_size))
        img_L = img_L[rnd_h:rnd_h + self.L_size, rnd_w:rnd_w + self.L_size]

        # --------------------------------
        # crop corresponding H patch
        # --------------------------------
        rnd_h_H, rnd_w_H = int(rnd_h * self.sf), int(rnd_w * self.sf)
        img_H = img_H[rnd_h_H:rnd_h_H + self.patch_size, rnd_w_H:rnd_w_H + self.patch_size]

        # --------------------------------
        # augmentation - flip and/or rotate
        # --------------------------------
        
        mode = random.randint(0, 7)
        img_L, img_H = util.augment_img(img_L, mode=mode), util.augment_img(img_H, mode=mode)
        
        # ------------------------------------
        # L/H pairs, HWC to CHW, numpy to tensor
        # ------------------------------------
        img_H, img_L = util.single2tensor3(img_H), util.single2tensor3(img_L)

        if L_path is None:
            L_path = H_path

        return {'L': img_L, 'H': img_H, 'L_path': L_path, 'H_path': H_path}

    def __len__(self):
        return len(self.paths_L)


#### Create Dataset and Dataloader

In [11]:
train_set = DatasetSR()
#train_sampler = DistributedSampler(train_set, shuffle=True, drop_last=True)
train_loader = DataLoader(train_set,
                          batch_size=8,
                          shuffle=True,
                          drop_last=True,
                          num_workers = 4,
                          pin_memory=True)

### Code for IACNN

In [12]:
import torch
import torch.nn as nn
#import tensorflow as tf
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR

import torchvision
class IACNN(nn.Module):
    def __init__(self):
        super(IACNN, self).__init__()
        #Conv_Head contains the 1st two convolutional layers for ICANN
        self.Conv_Head = nn.Sequential(nn.Conv2d(in_channels= 1, out_channels= 64, kernel_size=(7,7), stride=(1,1), padding=(0,0)),
                                      nn.ReLU(),
                                      nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(1,1), stride=(1,1), padding=(0,0)),
                                      nn.ReLU(),
                                      nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride=(1,1), padding=(4,4)),
                                      nn.ReLU()
                                      ) 
            
        
        #Inception Blocks
        self.Inception_1 = nn.Sequential(
            nn.Conv2d(in_channels= 64, out_channels= 8, kernel_size=(1,1), stride=(1,1), padding=(0,0)),
            nn.ReLU()
        )
        
        self.Inception_2 = nn.Sequential(
            nn.Conv2d(in_channels= 64, out_channels= 32, kernel_size=(1,1), stride=(1,1), padding=(0,0)),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels= 32, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.ReLU()
        )
        
        self.Inception_3 = nn.Sequential(
            nn.Conv2d(in_channels= 64, out_channels= 16, kernel_size=(1,1), stride=(1,1), padding=(0,0)),
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels= 16, kernel_size=(5,5), stride=(1,1), padding=(2,2)),
            nn.ReLU()
        )
        
        self.Inception_4 = nn.Sequential(
            nn.Conv2d(in_channels= 64, out_channels= 8, kernel_size=(1,1), stride=(1,1), padding=(0,0)),
            nn.ReLU(),
            nn.Conv2d(in_channels=8, out_channels= 8, kernel_size=(7,7), stride=(1,1), padding=(3,3)),
            nn.ReLU(),
        )
        
        #Conv_Tail contains the last convolutional layer for ICANN
        self.Conv_Tail = nn.Sequential(nn.Conv2d(in_channels= 64, out_channels= 1, kernel_size=(5,5), stride=(1,1), padding=(2,2))) 
        
        
        
        
        
        
        
    def forward(self, x):
        First_Conv_Output = self.Conv_Head(x)                                                                     #Pass Through 1st 3 Conv Layers
        
        Inception_1 = self.Inception_1(First_Conv_Output)                                                         ###############################
        Inception_2 = self.Inception_2(First_Conv_Output)                                                         #                             #
        Inception_3 = self.Inception_3(First_Conv_Output)                                                         #      First Inception        #
        Inception_4 = self.Inception_4(First_Conv_Output)                                                         ###############################
        
        First_Incpetion_Output = torch.cat([Inception_1,Inception_2, Inception_3, Inception_4], dim=1)      
        
        
        
        Inception_1 = self.Inception_1(First_Incpetion_Output)                                                    ###############################
        Inception_2 = self.Inception_2(First_Incpetion_Output)                                                    #                             #
        Inception_3 = self.Inception_3(First_Incpetion_Output)                                                    #     Second Inception        #
        Inception_4 = self.Inception_4(First_Incpetion_Output)                                                    ###############################
        
        Final_Incpetion_Output = torch.cat([Inception_1,Inception_2, Inception_3, Inception_4], dim=1)
        
        Final_Output = self.Conv_Tail(Final_Incpetion_Output)
        
        
        
       
        return Final_Output

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = IACNN()
model.to(device)
error = nn.L1Loss()

learning_rate = 1e-4 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # Code to use the Adam Optimizer
print(model)

IACNN(
  (Conv_Head): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(4, 4))
    (5): ReLU()
  )
  (Inception_1): Sequential(
    (0): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
  )
  (Inception_2): Sequential(
    (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (Inception_3): Sequential(
    (0): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): ReLU()
  )
  (Inception_4): Sequential(
    (0): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(8, 8, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (3): ReLU()
  )
  (Conv_Tail): Sequen

In [16]:
# Uncomment to start from a pre-saved check-point. Below is a format that can be used for loading models and optimizers 
'''
#IAM_3 = IAM_Net()
saved_state_dict = torch.load('/notebooks/IACNN_Files/models_JPEG_20/IACNN_JPEG_20_100000.pth')
model.load_state_dict(saved_state_dict)
#IAM_3 = IAM_3.to(device)
model.train()

#Load IAM Branch_1 Optimizer Checkpoint
saved_state_dict = torch.load('/notebooks/JPEG_Gray_IDPD_Files/models_1_channel/Optimizer300.pth')
optimizer.load_state_dict(saved_state_dict)
'''

"\n\n#Load IAM Branch_1 Optimizer Checkpoint\nsaved_state_dict = torch.load('/notebooks/JPEG_Gray_IDPD_Files/models_1_channel/Optimizer300.pth')\noptimizer.load_state_dict(saved_state_dict)\n"

#### Create a Logfile

In [17]:
import logging
logging.basicConfig(
        level=logging.DEBUG,
        format="%(asctime)s %(levelname)s %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
        filename="/notebooks/IACNN_Files/logs/train_IACNN_JPEG_10.log",
    )

In [ ]:
from tqdm.auto import tqdm
num_epochs = 4000
count = 0 * 450
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.train()


for epoch in range(1,num_epochs): 
    for idx, train_data in tqdm(enumerate(train_loader)):
        labels = []
        L = train_data['L'].to(device)
        H = train_data['H'].to(device)
        
        
        
        
        
        
        output = model(L)
        
        

        loss = error(output, H)

        # Initializing a gradient as 0 so there is no mixing of gradient among the batches
        optimizer.zero_grad()
        

        #Propagating the error backward
        loss.backward()

        # Optimizing the parameters
        optimizer.step()
        
        count += 1

    if (count % 500 == 0):
        logging.info("Iteration: {}, Loss: {}%, epoch: {}, Learning Rate: {}".format(count, loss.data,epoch, learning_rate))
        #scheduler.get_last_lr(), Learning Rate: {},
    if (count % 10000 == 0 and epoch != 0) or epoch ==3999 :
        torch.save(model.state_dict(), os.path.join("/notebooks/IACNN_Files/models_JPEG_10", 'IACNN_JPEG_10_' + str(count) + '.pth'))
        torch.save(optimizer.state_dict(), os.path.join("/notebooks/IACNN_Files/models_JPEG_10", 'Optimizer_' + str(count) + '.pth'))

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]